# 1. Setup Mario

In [2]:
!pip install gym_super_mario_bros==7.3.0 nes_py

  Using cached gym_super_mario_bros-7.3.0-py2.py3-none-any.whl (198 kB)
  Using cached nes_py-8.2.1.tar.gz (77 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl (14.8 MB)
  Using cached pyglet-1.5.21-py3-none-any.whl (1.1 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Created wheel for nes_py: filename=nes_py-8.2.1-cp311-cp311-win_amd64.whl size=45856 sha256=bb9ef007f91cc0824dc50018514bdb3fd802967e6360ad8d163696558427f121
  Stored in directory: c:\users\91701\appdata\local\pip\cache\whe

In [6]:
# # Import the game
# import gym_super_mario_bros
# # Import the Joypad wrapper
# from nes_py.wrappers import JoypadSpace
# # Import the SIMPLIFIED controls
# from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [11]:
!pip install gym --upgrade

In [14]:
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# Create the Super Mario Bros environment
env = gym_super_mario_bros.make('SuperMarioBros-v3')

# Wrap the environment with the JoypadSpace wrapper using the simplified controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Reset the environment
obs = env.reset()

# Loop through each frame in the game
done = False
while not done:
    # Choose a random action
    action = env.action_space.sample()
    
    # Perform the action in the environment
    step_result = env.step(action)
    if isinstance(step_result, tuple) and len(step_result) == 4:
        obs, reward, done, info = step_result
    else:
        obs, reward, done, info = step_result[0], step_result[1], step_result[2], {}

# Close the environment
env.close()


ValueError: not enough values to unpack (expected 5, got 4)

In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v3')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

C:\Users\91701\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [5]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000): 
    # Start the game to begin with 
    if done: 
        # Start the gamee
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

C:\Users\91701\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
C:\Users\91701\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
C:\Users\91701\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


ValueError: not enough values to unpack (expected 5, got 4)

# 2. Preprocess Environment

In [ ]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]

In [ ]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

# 3. Train the RL Model

In [ ]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

# 4. Test it Out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()